In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
input_width=28
input_hight=28
input_channel=1
input_pixel=784
stride_con1=1
stride_con2=1

n_con1=32
n_con2=64
con1_k=5
con2_k=5
max_pool1_k=2
max_pool2_k=2

n_hidden=1024
n_out=10

input_to_hidden=(input_width//max_pool1_k*max_pool2_k) *(input_hight//max_pool1_k*max_pool2_k) * n_con2

In [5]:
weight={
    "wc1":tf.Variable(tf.random_normal([con1_k,con1_k,input_channel,n_con1])),
    "wc2":tf.Variable(tf.random_normal([con2_k,con2_k,n_con1,n_con2])),
    "wh1":tf.Variable(tf.random_normal([input_to_hidden,n_hidden])),
    'out':tf.Variable(tf.random_normal([n_hidden,n_out])),
                      
}
bais={
    'bcn1':tf.Variable(tf.random_normal([n_con1])),
    'bcn2':tf.Variable(tf.random_normal([n_con2])),
    'bch':tf.Variable(tf.random_normal([n_hidden])),
    'bout':tf.Variable(tf.random_normal([n_out]))
}

In [6]:
def conn(x,weights,bais,strides=1):
    out=tf.nn.conv2d(x,weights,padding="SAME",strides=[1,strides,strides,1])
    out=tf.nn.bias_add(out,bais)
    out=tf.nn.relu(out)
    return out

def maxpool(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [7]:
def cnn(x,weights,bais):
    x=tf.reshape(x,shape=[-1,input_hight,input_width,input_channel])
    conv1=conn(x,weight['wc1'],bais['bcn1'])
    conv_pool1=maxpool(conv1,max_pool1_k)
    
    conv2=conn(conv_pool1,weight['wc2'],bais['bcn2'])
    conv_pool2=maxpool(conv2,max_pool2_k)
    
    max2_out=tf.reshape(conv_pool2,shape=[-1,input_to_hidden])
    
    hidden_before_activation=tf.add(tf.matmul(max2_out,weight['wh1']),bais['bch'])
    hidden_output=tf.nn.relu(hidden_before_activation)
    
    output=tf.add(tf.matmul(hidden_output,weight['out']),bais['bout'])
    return output
    

In [8]:
x=tf.placeholder("float",[None,input_pixel])
y=tf.placeholder(tf.int32,[None,n_out])
pred=cnn(x,weight,bais)

In [9]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

In [10]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)

In [11]:
se=tf.Session()
se.run(tf.global_variables_initializer())

In [12]:
for i in range(50):
    c,mi=se.run([cost,optimize],feed_dict={x:mnist.test.images,y:mnist.test.labels})

InvalidArgumentError: logits and labels must be broadcastable: logits_size=[625,10] labels_size=[10000,10]
	 [[node softmax_cross_entropy_with_logits (defined at /Users/sahilsagar/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'softmax_cross_entropy_with_logits':
  File "/Users/sahilsagar/opt/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/sahilsagar/opt/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sahilsagar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sahilsagar/opt/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/sahilsagar/opt/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()


In [4]:
import numpy as np
train=np.reshape(mnist.train.images,(55000,28,28,1))
test=np.reshape(mnist.test.images,(10000,28,28,1))

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten,Dropout

Using TensorFlow backend.


In [6]:
model=Sequential()
model.add(Conv2D(10,kernel_size=(5,5),strides=1,padding="SAME"))
model.add(MaxPool2D(pool_size=2))
model.add(Conv2D(10,kernel_size=(5,5),strides=1,padding="SAME"))
model.add(MaxPool2D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10,activation="sigmoid"))

In [7]:
model.compile(loss="binary_crossentropy",metrics=["accuracy"],optimizer="adam")

In [8]:
model.fit(train,mnist.train.labels,validation_data=(test,mnist.test.labels),epochs=2,batch_size=50)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 55000 samples, validate on 10000 samples
Epoch 1/2





55000/55000 [==============================] - 97s 2ms/step - loss: 0.1123 - acc: 0.9604 - val_loss: 0.0375 - val_acc: 0.9874
Epoch 2/2
55000/55000 [==============================] - 93s 2ms/step - loss: 0.0494 - acc: 0.9831 - val_loss: 0.0246 - val_acc: 0.9924


In [35]:
mnist.test.images.shape

(10000, 784)

AttributeError: 'Sequential' object has no attribute 'loss'